In [2]:
!which python

/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/bin/python


In [3]:
!source activate py36

/usr/bin/sh: line 0: source: activate: file not found


In [4]:
import sys

#sys.path.append('/home/faske/g/anaconda3/envs/py34/lib/python3.4/site-packages')
sys.path.append('/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/data/gpfs/assoc/parchmanlab/tfaske/ipynb/include_utils")

import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import random
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
import re
from itertools import chain
#import Levenshtein as lv

In [43]:
from os import environ

In [5]:
fq_dir = '/data/gpfs/assoc/denovo/tfaske/connie/ch2/fastq/edit'

In [6]:
cd $fq_dir

/data/gpfs/assoc/denovo/tfaske/connie/ch2/fastq/edit


In [16]:
fq_files = []
files = !find . -type f -name '*.F.fq.gz'
files = [os.path.abspath(x) for x in files]
for x in files:
    fq_files.append(x)
fq_files = sorted(fq_files)
len(fq_files),fq_files[0]

(210, '/data/gpfs/assoc/denovo/tfaske/connie/ch2/fastq/edit/HY_BM_01.F.fq.gz')

#### Count  reads for all fq

In [84]:
fq_names = []
fq_reads = []
for f in fq_files:
    environ['f'] = f
    r = !zcat $f | awk '{s++}END{print s/4}'
    name = f.split('.F')[0]
    name = name.split('/')[10]
    #print(name)
    #print(int(r.n))
    fq_names.append(name)
    fq_reads.append(int(r.n))

In [85]:
len(fq_reads), fq_reads[:5]

(210, [815375, 1603815, 2126509, 1564018, 1207634])

In [86]:
len(fq_names), fq_names[0]

(210, 'HY_BM_01')

In [87]:
read_dict = {"fq":fq_names,'reads':fq_reads}
read_df = pd.DataFrame(read_dict)
read_df.head()
#len(cov_df)

,fq,reads
0,HY_BM_01,815375
1,HY_BM_03,1603815
2,HY_BM_04,2126509
3,HY_BM_05,1564018
4,HY_DT_02,1207634


In [89]:
read_out = os.path.join(fq_dir,'fq_read.csv')
read_df.to_csv(path_or_buf=read_out)

#### Calculate outliers and max of non outlier group

In [94]:
read_df.sort_values('reads',ascending=False,inplace=True)
read_df[:20]

,fq,reads
106,PU_LF_99,16013949
178,RI_LF_01,15052999
79,PU_LF_2,14776709
179,RI_LF_04,14642277
182,RI_LF_07,14528108
146,RI_DT_06,14401868
184,RI_LF_09,14204223
145,RI_DT_05,13985912
181,RI_LF_06,13753739
105,PU_LF_5,13538752


### Top 15 are outliers, next max is RI_DT_31	3024291

In [121]:
fq_edit = list(read_df.fq[:15])
read_max = int(read_df['reads'][ (read_df.fq == 'RI_DT_31')]*4)

In [123]:
fq_edit, read_max

(['PU_LF_99',
  'RI_LF_01',
  'PU_LF_2',
  'RI_LF_04',
  'RI_LF_07',
  'RI_DT_06',
  'RI_LF_09',
  'RI_DT_05',
  'RI_LF_06',
  'PU_LF_5',
  'RI_LF_10',
  'PU_DT_42',
  'RI_DT_08',
  'RI_LF_08',
  'PU_DT_40'],
 12097164)

In [130]:
for f in fq_edit:
    f_qz = f + '.F.fq.gz'
    out_f = f + '_edit.F.fq'
    environ['f_qz'] = f_qz
    environ['read_max'] = str(read_max)
    environ['out_f'] = out_f
    !zcat $f_qz | head -n $read_max > $out_f


gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe


#### move correct files to ddocent, do not include VA pines